In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TesteNotebook").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/28 16:15:04 WARN Utils: Your hostname, LAPTOP-CO0U45G1, resolves to a loopback address: 127.0.1.1; using 172.20.235.103 instead (on interface eth0)
25/07/28 16:15:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/28 16:15:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/28 16:15:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [3]:
carros_temp = spark.read.csv('/home/robson/curso_spark_pyspark_udemy/Carros.csv', 
                             inferSchema=True, 
                             header=True,
                             sep=';')


In [4]:
carros_temp.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [5]:
carros = carros_temp.select("Consumo", "Cilindros", "Cilindradas", "HP")
carros.show()

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
|    181|        6|        225|105|
|    143|        8|        360|245|
|    244|        4|       1467| 62|
|    228|        4|       1408| 95|
|    192|        6|       1676|123|
|    178|        6|       1676|123|
|    164|        8|       2758|180|
|    173|        8|       2758|180|
|    152|        8|       2758|180|
|    104|        8|        472|205|
|    104|        8|        460|215|
|    147|        8|        440|230|
|    324|        4|        787| 66|
|    304|        4|        757| 52|
|    339|        4|        711| 65|
+-------+---------+-----------+---+
only showing top 20 rows


In [6]:
vec_caracteristicas = VectorAssembler(
    inputCols=["Consumo", "Cilindros", "Cilindradas"],
    outputCol="caracteristicas"
)

In [7]:
vec_carrosTreino = vec_caracteristicas.transform(carros)

In [8]:
vec_carrosTreino.show()

+-------+---------+-----------+---+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|
+-------+---------+-----------+---+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|
|     21|        6|        160|110|  [21.0,6.0,160.0]|
|    228|        4|        108| 93| [228.0,4.0,108.0]|
|    214|        6|        258|110| [214.0,6.0,258.0]|
|    187|        8|        360|175| [187.0,8.0,360.0]|
|    181|        6|        225|105| [181.0,6.0,225.0]|
|    143|        8|        360|245| [143.0,8.0,360.0]|
|    244|        4|       1467| 62|[244.0,4.0,1467.0]|
|    228|        4|       1408| 95|[228.0,4.0,1408.0]|
|    192|        6|       1676|123|[192.0,6.0,1676.0]|
|    178|        6|       1676|123|[178.0,6.0,1676.0]|
|    164|        8|       2758|180|[164.0,8.0,2758.0]|
|    173|        8|       2758|180|[173.0,8.0,2758.0]|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|
|    104|        8|        472|205| [104.0,8.0,472.0]|
|    104| 

In [9]:
reglin = LinearRegression(
    featuresCol="caracteristicas",
    labelCol="HP",
    maxIter=10,
    regParam=0.3,
    elasticNetParam=0.8
    )

In [10]:
modelo = reglin.fit(vec_carrosTreino)

25/07/28 16:21:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [12]:
from pyspark.ml import Pipeline

In [13]:
pipeline = Pipeline(stages=[vec_caracteristicas, reglin])

In [15]:
pipelineModel = pipeline.fit(carros)

In [16]:
previsao = pipelineModel.transform(carros)

In [17]:
previsao.show(truncate=False)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas|HP |caracteristicas   |prediction        |
+-------+---------+-----------+---+------------------+------------------+
|21     |6        |160        |110|[21.0,6.0,160.0]  |161.97644881051687|
|21     |6        |160        |110|[21.0,6.0,160.0]  |161.97644881051687|
|228    |4        |108        |93 |[228.0,4.0,108.0] |82.66378378666553 |
|214    |6        |258        |110|[214.0,6.0,258.0] |141.75976237910498|
|187    |8        |360        |175|[187.0,8.0,360.0] |202.54523497777276|
|181    |6        |225        |105|[181.0,6.0,225.0] |145.34509861662772|
|143    |8        |360        |245|[143.0,8.0,360.0] |206.97734306141297|
|244    |4        |1467       |62 |[244.0,4.0,1467.0]|70.29314501252023 |
|228    |4        |1408       |95 |[228.0,4.0,1408.0]|72.3719132645337  |
|192    |6        |1676       |123|[192.0,6.0,1676.0]|132.74976072832283|
|178    |6        |1676       |123|[17